In [1]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import string
import pandas as pd
lemmatizer = WordNetLemmatizer()

In [2]:
train_data_df = pd.read_csv('training_twitter_x_y_train.csv', delimiter=',')
test_data_df = pd.read_csv('test_twitter_x_test.csv', delimiter=',')

In [3]:
del train_data_df['airline_sentiment_gold']
del train_data_df['negativereason_gold']
del train_data_df['tweet_coord']

del test_data_df['airline_sentiment_gold']
del test_data_df['negativereason_gold']
del test_data_df['tweet_coord']

In [4]:
train_data_df

,tweet_id,airline_sentiment,airline,name,retweet_count,text,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,ColeyGirouard,0,"@SouthwestAir I am scheduled for the morning, ...",2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,WalterFaddoul,0,@SouthwestAir seeing your workers time in and ...,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,LocalKyle,0,@united Flew ORD to Miami and back and had gr...,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,amccarthy19,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,J_Okayy,0,@united so our flight into ORD was delayed bec...,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)
...,...,...,...,...,...,...,...,...,...
10975,569934458364813313,neutral,American,Cottopanama85,0,@AmericanAir followback,2015-02-23 10:58:58 -0800,"ohio,panama",NaN
10976,568564006329434113,positive,United,PaulBEsteves,0,@united thanks for the help. Wish the phone re...,2015-02-19 16:13:17 -0800,Brooklyn,Eastern Time (US & Canada)
10977,569643648910028801,negative,US Airways,runfixsteve,0,@usairways the. Worst. Ever. #dca #customerser...,2015-02-22 15:43:24 -0800,"St. Augustine, Florida",NaN
10978,568864981917110272,negative,US Airways,CLChicosky,0,@nrhodes85: look! Another apology. DO NOT FLY ...,2015-02-20 12:09:15 -0800,NaN,NaN


In [5]:
def make_lists_train(df):
    data = []
    for i in range(len(df)) :
        raw_data = df.iloc[i, 0:1].tolist() + df.iloc[i, 2:].tolist()
        category = df.iloc[i, 1]
        a = []
        for j in raw_data:
            if not pd.isna(j):
                words = word_tokenize(str(j))
                for k in words:
                    a.append(str(k))
        data.append((a, category))
    
    return data

In [6]:
def make_lists_test(df):
    data = []
    for i in range(len(df)) :
        raw_data = df.iloc[i, :].tolist()
        a = []
        for j in raw_data:
            if not pd.isna(i):
                words = word_tokenize(str(j))
                for k in words:
                    a.append(str(k))
        data.append(a)
        
    return data

In [7]:
train_data = make_lists_train(train_data_df)
test_data = make_lists_test(test_data_df)

In [8]:
stops = stopwords.words('english')
punc = (string.punctuation)
stops += punc

In [9]:
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [63]:
def clean_data(words):
    output_words=[]
    for w in words:
        if w.lower() not in stops:
            pos = pos_tag([w])
            clean_words = lemmatizer.lemmatize(w, pos=get_simple_pos(pos[0][1]))
            output_words.append(clean_words.lower())

    return output_words

In [80]:
train_doc = [(clean_data(document), category) for document, category in train_data]
train_doc

[(['567900433542488064',
   'southwest',
   'coleygirouard',
   '0',
   'southwestair',
   'schedule',
   'morning',
   '2',
   'day',
   'fact',
   'yes..not',
   'sure',
   'even',
   'flight',
   'one',
   'cancelled',
   'flightled',
   '2015-02-17',
   '20:16:29',
   '-0800',
   'washington',
   'd.c',
   'atlantic',
   'time',
   'canada'],
  'negative'),
 (['569989168903819264',
   'southwest',
   'walterfaddoul',
   '0',
   'southwestair',
   'see',
   'worker',
   'time',
   'time',
   'go',
   'beyond',
   'love',
   'fly',
   'guy',
   'thank',
   '2015-02-23',
   '14:36:22',
   '-0800',
   'indianapolis',
   'indiana',
   'usa',
   'central',
   'time',
   'us',
   'canada'],
  'positive'),
 (['568089179520954368',
   'united',
   'localkyle',
   '0',
   'united',
   'flew',
   'ord',
   'miami',
   'back',
   'great',
   'crew',
   'service',
   'leg',
   'thanks',
   '2015-02-18',
   '08:46:29',
   '-0800',
   'illinois',
   'central',
   'time',
   'us',
   'canada'],
  

In [81]:
test_doc = [clean_data(document) for document in test_data]
test_doc

[['569682010270101504',
  'american',
  'zsalim03',
  '0',
  'americanair',
  'car',
  'gng',
  'dfw',
  'pulled',
  '1hr',
  'ago',
  'icy',
  'road',
  'on-hold',
  'aa',
  'since',
  '1hr',
  'ca',
  "n't",
  'reach',
  'arpt',
  'aa2450',
  'wat',
  '2',
  '2015-02-22',
  '18:15:50',
  '-0800',
  'texas',
  'central',
  'time',
  'us',
  'canada'],
 ['569608307184242688',
  'american',
  'sa_craig',
  '0',
  'americanair',
  'plane',
  '’',
  'land',
  'identical',
  'bad',
  'condition',
  'grk',
  'accord',
  'metars',
  '2015-02-22',
  '13:22:57',
  '-0800',
  'college',
  'station',
  'tx',
  'central',
  'time',
  'us',
  'canada'],
 ['567879304593408001',
  'southwest',
  'danachristos',
  '1',
  'southwestair',
  'ca',
  "n't",
  'believe',
  'many',
  'pay',
  'customer',
  'left',
  'high',
  'dry',
  'reason',
  'flight',
  'cancelled',
  'flightlations',
  'monday',
  'bdl',
  'wow',
  '2015-02-17',
  '18:52:31',
  '-0800',
  'ct',
  'eastern',
  'time',
  'us',
  'canad

In [82]:
categories = [category for document, category in train_doc]
x_train_text = [' '.join(document) for document, category in train_doc]

x_test_text = [' '.join(document) for document in test_doc]

In [83]:
len(x_train_text), len(x_test_text)

(10980, 3660)

In [84]:
#splitting data
x_train, x_test_self, y_train, y_test_self = train_test_split(x_train_text, categories)

In [85]:
len(x_train), len(y_train)

(8235, 8235)

In [86]:
count_vec = CountVectorizer(max_features = 2500, ngram_range = (1,2), max_df = 0.85)

In [87]:
x_train_features = count_vec.fit_transform(x_train).todense() ## train data
x_test_self = count_vec.transform(x_test_self).todense()   ## I split the train data for testing purposes
x_test_features = count_vec.transform(x_test_text).todense() ## actual test data

In [88]:
x_train_features.shape, x_test_features.shape, x_test_self.shape

((8235, 2500), (3660, 2500), (2745, 2500))

In [89]:
len(count_vec.get_feature_names())

2500

In [90]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report

In [91]:
#grid = { 'C' : [1e2, 1e3, 5e3, 1e4, 1e5], 'gamma' : [1e-3, 5e-4, 1e-4, 5e-3, 1e-2]}
svc = SVC(C = 1e2, gamma = 1e-2, decision_function_shape='ovo')
svc.fit(x_train_features, y_train)

SVC(C=100.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma=0.01, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

grid = { 'C' : [1e2, 1e3, 5e3, 1e4, 1e5], 'gamma' : [1e-3, 5e-4, 1e-4, 5e-3, 1e-2]}
parameters = GridSearchCV(svc, grid)
parameters.fit(x_train_features, y_train)
parameters.best_estimator_

In [315]:
y_pred_self = svc.predict(x_test_self)

In [93]:
y_pred = svc.predict(x_test_features)

In [92]:
svc.score(x_test_self, y_test_self)

0.7482695810564663

In [94]:
import numpy as np
np.savetxt("Y_twitter_test.csv", y_pred, fmt='%s')

1e2, 1e-3 = 0.627, 06273
1e2, 5e-4 = 0.625
1e2, 1e-4 = 0.625
1e2, 5e-3 = 0.631
1e2, 1e-2 = 0.6331
1e2, 1e-2 (ovo) = 0.6338
1e3, 1e-3 = 0.6273
1e3, 1e-4 = 0.6273
1e3, 1e-5 = 0.6251
1e3, 5e-3 = 0.6331
5e3, 5e-3 = 0.6255

In [287]:
accuracy_score(y_test_self, y_pred)

0.6313296903460838

In [34]:
rf = RandomForestClassifier(random_state=0, min_samples_split=3, min_samples_leaf=2, max_features='log2')
rf.fit(x_train_features, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='log2',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=3,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [35]:
rf.score(x_train_features, y_train), rf.score(x_test_self, y_test_self)

(0.7241044323011536, 0.6998178506375228)

In [36]:
y_pred_rf = rf.predict(x_test_features)

In [344]:
type(y_pred_rf)

numpy.ndarray

In [37]:
np.savetxt('y_pred_twitter_rf.csv', y_pred_rf, fmt='%s')